In [6]:
from tkinter import messagebox
import customtkinter as ctk
import tkinter as tk
import subprocess
import time
import wmi
import os

default_path = "E:\\Driver"
default_path = os.path.expanduser("~\Desktop\Driver")
print(default_path)
#default_path = os.path.expanduser(("~"))

font = ('Dubai Medium', 15)
font1 = ('Dubai Medium', 15, 'bold')
font2= ('Dubai Medium', 16)

driver_search_keywords = {
    "Intel® Chipset": ["chipset"],
    "Intel® Management Engine": ["management engine"],
    "Intel® Graphics": ["graphics"],
    "Intel® Serial IO": ["serial io"],
    "Intel® Network": ["network"],
    "Intel® GNA": ["gna"],
    "Intel® HID": ["hid"],
    "AcpiSmbus": ["acpismbus"],
    "KSC20": ["ksc20"],
    "Core-Temp": ["core-temp"],
    "bitwindows": ["bitwindows"],
    "FUSBCAN": ["fusbcam"]
}

def start_drag(event, window):
    window.x_offset = event.x
    window.y_offset = event.y

def on_drag(event, window):
    x = window.winfo_pointerx() - window.x_offset
    y = window.winfo_pointery() - window.y_offset
    window.geometry(f"+{x}+{y}")

def real_close_window():
    messagebox.showwarning("Hint", "Please choose one of the modules")

def find_folder(driver_name, full_path):
    keywords = driver_search_keywords.get(driver_name, [driver_name.lower()])
    for item in os.listdir(full_path):
        item_path = os.path.join(full_path, item)
        if any(keyword in item.lower() for keyword in keywords):
            return item_path
    return None

def install_driver(driver_path, driver_name):
    silent_drivers = ["KSC20", "Core-Temp", "bitwindows"]

    if os.path.isfile(driver_path):
        command = [driver_path, "/SILENT"] if driver_name in silent_drivers else [driver_path, "-s"]
        print(f"Running: {' '.join(command)}")
        # subprocess.run(command, check=True)
    elif os.path.isdir(driver_path):
        for item in os.listdir(driver_path):
            item_path = os.path.join(driver_path, item)
            if item.lower().endswith(".exe"):
                command = [item_path, "/SILENT"] if driver_name in silent_drivers else [item_path, "-s"]
                print(f"Running: {' '.join(command)}")
                # subprocess.run(command, check=True)
                break
            elif item.lower().endswith(".inf"):
                print(f"Installing INF driver: {item_path}")
                # subprocess.run(["pnputil", "/add-driver", item_path, "/install"], check=True)
                break
    
def check_device_manager_for_issues():
    wmi_service = wmi.WMI()
    problem_device = []

    for device in wmi_service.Win32_PnPEntity():
        if device.ConfigManagerErrorCode and device.ConfigManagerErrorCode != 0:
            problem_device.append(f"{device.Name} - Error Code: {device.ConfigManagerErrorCode}")
    if problem_device:
        messagebox.showwarning("Device Manager Issues", "The following devices have issues:\n" + "\n".join(problem_device))
    else:
        messagebox.showinfo("Device Manager", "All devices are working properly with no issues")

            
def open_driver_window():
    root.withdraw()
    selected = selected_option.get()
    full_path = os.path.join(default_path, selected)
    print(full_path)

    driver_window = ctk.CTkToplevel(root)
    driver_window.title("Drivers Selection")
    driver_window.geometry('600x700')
    driver_window.configure(fg_color='#cce6ff')
    driver_window.overrideredirect(True)

    def real_close_window2():
        driver_window.destroy()
        root.quit()

    second_bar = ctk.CTkFrame(driver_window, height=20, fg_color="#003359")
    second_bar.pack(fill="x", side="top")
    
    second_label = ctk.CTkLabel(second_bar, text="Drivers Selection", font=font2, text_color="white")
    second_label.pack(side="left", padx=10)

    second_bar.bind("<Button-1>", lambda event: start_drag(event, driver_window))
    second_bar.bind("<B1-Motion>", lambda event: on_drag(event, driver_window))

    driver_custom_exit_button = ctk.CTkButton(second_bar, text="X", command=real_close_window2, width=30, height=30, corner_radius=15, fg_color="red")
    driver_custom_exit_button.pack(side="right", padx=5)

    driver_window.after(200, lambda : driver_window.iconbitmap('Kontron.ico'))

    checkbox_frame = ctk.CTkFrame(driver_window, fg_color="#edf2f2", corner_radius=10)
    checkbox_frame.pack(padx=15, side="left")

    version_display_frame = ctk.CTkFrame(driver_window, fg_color="#f0f0f0", corner_radius=10, width=350)
    version_display_frame.pack(side="right", fill="both", expand=False, padx=(0, 20), pady=20)  #fill="both", expand=False, padx=(0, 20), pady=20, 
    
    version_title = ctk.CTkLabel(version_display_frame, text="Drivers Version", font=font1, text_color="black")
    version_title.pack(anchor="n", pady=(10, 5))
    
    version_text = ctk.CTkTextbox(version_display_frame, width=320, height=400, font=font)
    version_text.pack(fill="both", expand=True, padx=10, pady=10) 

    driver_options = ["Intel® Chipset", "Intel® Management Engine", "Intel® Graphics", 
                      "Intel® Serial IO", "Intel® Network", "Intel® GNA", "Intel® HID", 
                      "AcpiSmbus", "KSC20", "Core-Temp", "bitwindows", "FUSBCAN"]
    selected_drivers = {opt: ctk.IntVar(value=0) for opt in driver_options}

    for row, option in enumerate(driver_options):
        checkbox = ctk.CTkCheckBox(
            checkbox_frame, text=option, font=font,
            variable=selected_drivers[option],
            fg_color="#175e22", hover_color="#5a8a96",
            border_color="#3a728a", corner_radius=6
        )
        checkbox.pack(anchor="w", pady=10)

    def check_driver_version():
        w = wmi.WMI()
        result_text = ""
        keywords = ["Ethernet","Graphics"]
    
        for device in w.Win32_PnPSignedDriver():
            if device.DeviceName:
                for keyword in keywords:
                    if keyword in device.DeviceName:
                        result_text += f"Device: {device.DeviceName}\n"
                        result_text += f"Driver Version: {device.DriverVersion}\n"
                        result_text += "-" * 40 + "\n"
        version_text.insert("1.0", result_text if result_text else "No matching drivers found.")

    def start_installation():
        try:
            for option, var in selected_drivers.items():
                if var.get() == 1:
                    folder_or_file = find_folder(option, full_path)
                    if folder_or_file:
                        try:
                            print(f"Installing {option} from {folder_or_file}")
                            install_driver(folder_or_file, option)
                            print(f"Successfully installed {option}")
                        except subprocess.CalledProcessError as e:
                            print(f"Failed to install {option}: {e}")
                            messagebox.showerror("Installation Error", f"Failed to install {option}: {e}")
                    else:
                        print(f"No folder or file found for {option}")
                        messagebox.showwarning("Driver Not Found", f"No folder or file found for {option}")
        except Exception as e:
            print(f"An unexpected error occurred: {e}")
            messagebox.showerror("Error", f"An unexpected error occurred: {e}")
            
        messagebox.showinfo("Installation Complete", "Selected drivers have been processed.")
        check_device_manager_for_issues()
        check_driver_version()
    
    button = ctk.CTkButton(checkbox_frame, command=start_installation, 
                           text="Install", font=font1, 
                           fg_color="#3a728a", hover_color="#79a6ad", 
                           text_color="white", corner_radius=10).pack(pady=10, padx=10) #command=start_installation,
    
root = ctk.CTk()
root.geometry('280x200')
root.iconbitmap('Kontron.ico')
root.configure(fg_color='#cce6ff')
root.overrideredirect(True)

title_bar = ctk.CTkFrame(root, height=20, fg_color="#003359")
title_bar.pack(fill="x", side="top")

title_label = ctk.CTkLabel(title_bar, text="Module Selection", font=font2, text_color="white")
title_label.pack(side="left", padx=10)

custom_exit_button = ctk.CTkButton(title_bar, text="X", command=real_close_window, width=30, height=30, corner_radius=15, fg_color="red")
custom_exit_button.pack(side="right", padx=5)

options = ["DRIVER-3.5-SBC-ADN", "DRIVER-2.5-SBC-ADN", "DRIVER-3.5-SBC-RPL"]
selected_option = ctk.StringVar(value="DRIVER-3.5-SBC-ADN")

for option in options:
    checkbox = ctk.CTkRadioButton(
        root, text=option, font=font, variable=selected_option,
        value=option, border_color="#003359", hover_color="#002d75",
        fg_color="#c1d6d0"
    )  
    checkbox.pack(anchor="center", pady=5)

ctk.CTkButton(root, text="Next", font=font1,command=open_driver_window,
              fg_color="#3a728a", hover_color="#79a6ad", text_color="white", 
              corner_radius=10).pack(pady=(20, 10), anchor="center")


title_bar.bind("<Button-1>", lambda event: start_drag(event, root))
title_bar.bind("<B1-Motion>", lambda event: on_drag(event, root))

root.mainloop()

C:\Users\Kontron/Desktop
C:\Users\Kontron/Desktop\DRIVER-2.5-SBC-ADN
